<a href="https://colab.research.google.com/github/yufengg/advent-of-code-2024/blob/master/AoC_2024_day_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: python code download advent of code file

import requests
import os
from datetime import datetime

def download_aoc_input(year, day, session_cookie):
  """Downloads the input file for a specific Advent of Code day.

  Args:
    year: The year of the challenge (e.g., 2023).
    day: The day of the challenge (e.g., 1).
    session_cookie: Your session cookie from the Advent of Code website.
  """

  url = f"https://adventofcode.com/{year}/day/{day}/input"
  headers = {
      "User-Agent": "github.com/yourusername/yourrepo (or your email)",  # Replace with your info
      "Cookie": f"session={session_cookie}"
  }

  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an exception for bad status codes

    # Create a directory for the year if it doesn't exist
    os.makedirs(str(year), exist_ok=True)

    filename = os.path.join(str(year), f"day{day}.txt")
    with open(filename, "w") as file:
      file.write(response.text)
    print(f"Successfully downloaded input for year {year}, day {day} to {filename}")
    return filename

  except requests.exceptions.RequestException as e:
    print(f"Error downloading input: {e}")
  except Exception as e:
    print(f"An unexpected error occurred: {e}")


# Example usage:  Replace with your actual values
year = 2024  #@param {type:"integer"}
day = 6 #@param {type:"integer"}
session_cookie = "53616c7465645f5fd6809f86933785dcec3c4f79b521758a796b0808898e32ff9a09d06dd15282b2a68340b7bf970ab66894fa0bf9271caa521bc5d5fc8df3b9" #@param {type:"string"}

input_filename = download_aoc_input(year, day, session_cookie)

Successfully downloaded input for year 2024, day 6 to 2024/day6.txt


In [30]:
import jax
import jax.numpy as jnp
import numpy as np

Need a matrix representation of the grid, probably a class.

Need a class member function to take a step, with parameter to make a turn to the right 90 degrees. This function will avoid obstacles marked by '#' (probably represented with 1, while valid tiles are zeros?).

during the walking, need to detect exit criteria when guard walks off-map.

For each step, return and record the (new) position in a set, so we can count it up.



In [6]:

def read_input(filename, parser=None):
  with open(filename, 'r') as file:
    lines = file.read().splitlines()
    if parser:
      lines = list(map(parser, lines))
  return lines


In [7]:
input_filename = f'{year}/day{day}.txt'
def parser(line):
  return line
  return tuple(map(int, line.split('|')))



# Test data

In [12]:
test_data = '''....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
'''
test_file = f'day{day}_test.txt'
with open(test_file, 'w') as file:
  lines = file.writelines(test_data)


In [117]:
lines = read_input(test_file, parser)


In [118]:
lines

['....#.....',
 '.........#',
 '..........',
 '..#.......',
 '.......#..',
 '..........',
 '.#..^.....',
 '........#.',
 '#.........',
 '......#...']

In [119]:
lines[0][3]

'.'

In [120]:
#location of the guard is?
for i in range(len(lines)):
  for j in range(len(lines[0])):
    if lines[i][j] == '^':
      guard_pos = (i,j)

In [121]:
guard_pos

(6, 4)

In [113]:
class Grid:
  def __init__(self, lines):
    self.lines = lines
    self.height = len(lines)
    self.width = len(lines[0])
    self.guard_pos = self.find_guard()
    self.walk_path = []
    self.direction = 'N'

  def __str__(self):
    return '\n'.join(self.lines)

  def find_guard(self):
    for i in range(self.height):
      for j in range(self.width):
        if self.lines[i][j] == '^':
          return np.array([i,j])

  def print_grid(self):
    current_grid = self.lines.copy()
    current_grid[self.guard_pos[0]] = current_grid[self.guard_pos[0]][:self.guard_pos[1]] + self.direction + current_grid[self.guard_pos[0]][self.guard_pos[1]+1:]
    # current_grid[self.guard_pos[0]][self.guard_pos[1]] = 'X'
    for line in current_grid:
      print(line)

  def direction_to_delta(self):
    if self.direction == 'N':
      return np.array([-1,0])
    elif self.direction == 'S':
      return np.array([1,0])
    elif self.direction == 'E':
      return np.array([0,1])
    elif self.direction == 'W':
      return np.array([0,-1])
    return self.direction

  def turn_90(self):
    if self.direction == 'N':
      self.direction = 'E'
    elif self.direction == 'E':
      self.direction = 'S'
    elif self.direction == 'S':
      self.direction = 'W'
    elif self.direction == 'W':
      self.direction = 'N'

  # just checks if it's on the grid
  def validate_step(self, coordinate):
    if coordinate[0] < 0 or coordinate[0] >= self.height:
      return False
    if coordinate[1] < 0 or coordinate[1] >= self.width:
      return False
    return True

  # default walk "forward", need to track direction
  # option to turn. Perhaps separate function that alters direction only?
  # need to detect obstacle (and then turn, and then step?)
  # Check for valid square before walking, since that's the end condition
  # return new position, direction, and also save historical path as a list of tuples.
  def step(self):
    potential_step = self.guard_pos + self.direction_to_delta()

    if not self.validate_step(potential_step):
      return False

    if self.lines[potential_step[0]][potential_step[1]] == '#':
      self.turn_90()
      return True

    self.guard_pos = potential_step
    self.walk_path.append(tuple((self.guard_pos[0], self.guard_pos[1])))
    return True # self.guard_pos


In [109]:
x = np.array([1,2])
y = np.array([1,3])
# np.equal(x,y)
all(np.equal(x,y))

False

In [93]:
g = Grid(lines)

In [76]:
g.find_guard()

array([6, 4])

In [36]:
g.direction_to_delta()

array([-1,  0])

In [37]:
g.find_guard() + g.direction_to_delta()


array([5, 4])

In [54]:
g.step()

array([4, 4])

In [94]:
while False != g.step():
  g.print_grid()

path = g.walk_path

....#.....
.........#
..........
..#.......
.......#..
....N.....
.#..^.....
........#.
#.........
......#...
....#.....
.........#
..........
..#.......
....N..#..
..........
.#..^.....
........#.
#.........
......#...
....#.....
.........#
..........
..#.N.....
.......#..
..........
.#..^.....
........#.
#.........
......#...
....#.....
.........#
....N.....
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
....#.....
....N....#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
....#.....
....E....#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
....#.....
.....E...#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
....#.....
......E..#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
....#.....
.......E.#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
....#.....

In [95]:
len(g.walk_path)

44

In [96]:
len(set(g.walk_path))

41

# Part 1

In [98]:
lines = read_input(input_filename)


In [99]:
g = Grid(lines)

while False != g.step():
  # g.print_grid()
  pass

path = g.walk_path



In [100]:
len(set(g.walk_path))

5329

# part 2

for each position, run the simulation and see if we ever return to the same location _and_ orientation (specifically, the starting location+orientation). If so, record it as viable.

In [285]:
class Grid:
  def __init__(self, lines, new_obstacle): # new_obstacle: [x,y]
    self.lines = lines
    self.height = len(lines)
    self.width = len(lines[0])
    self.original_guard_pos = self.find_guard()
    self.guard_pos = self.find_guard()
    self.walk_path = set()
    self.direction = 'N'
    self.new_obstacle = np.array(new_obstacle)
    self.loop_found = False


  def __str__(self):
    return '\n'.join(self.lines)

  def find_guard(self):
    for i in range(self.height):
      for j in range(self.width):
        if self.lines[i][j] == '^':
          return np.array([i,j])

  def print_grid(self):
    current_grid = self.lines.copy()
    current_grid[self.guard_pos[0]] = current_grid[self.guard_pos[0]][:self.guard_pos[1]] + self.direction + current_grid[self.guard_pos[0]][self.guard_pos[1]+1:]
    # current_grid[self.guard_pos[0]][self.guard_pos[1]] = 'X'
    for line in current_grid:
      print(line)

  def direction_to_delta(self):
    if self.direction == 'N':
      return np.array([-1,0])
    elif self.direction == 'S':
      return np.array([1,0])
    elif self.direction == 'E':
      return np.array([0,1])
    elif self.direction == 'W':
      return np.array([0,-1])
    return self.direction

  def turn_90(self):
    if self.direction == 'N':
      self.direction = 'E'
    elif self.direction == 'E':
      self.direction = 'S'
    elif self.direction == 'S':
      self.direction = 'W'
    elif self.direction == 'W':
      self.direction = 'N'

  # just checks if it's on the grid
  def validate_step(self, coordinate):
    if coordinate[0] < 0 or coordinate[0] >= self.height:
      return False
    if coordinate[1] < 0 or coordinate[1] >= self.width:
      return False
    return True

  # default walk "forward", need to track direction
  # option to turn. Perhaps separate function that alters direction only?
  # need to detect obstacle (and then turn, and then step?)
  # Check for valid square before walking, since that's the end condition
  # return new position, direction, and also save historical path as a list of tuples.
  def step(self):
    potential_step = self.guard_pos + self.direction_to_delta()

    # print(self.guard_pos)

    if not self.validate_step(potential_step):
      # print('***Exiting room')
      return False

    if self.lines[potential_step[0]][potential_step[1]] == '#' or all(potential_step == self.new_obstacle):
      self.turn_90()
      return True

    if tuple((potential_step[0], potential_step[1], self.direction)) in self.walk_path:
      self.loop_found = True
      print('***loop found', self.new_obstacle)
      return False # stop looping

    self.guard_pos = potential_step # this effectively takes the step

    self.walk_path.add(tuple((self.guard_pos[0], self.guard_pos[1], self.direction)))

    return True # self.guard_pos


In [276]:
def check_single_obstacle(obstacle):
  g = Grid(lines, obstacle)
  while False != g.step():
    # print(g.guard_pos, g.direction)
    pass

  if g.loop_found:
    return True

  # print(g.walk_path)
  return False

In [277]:
obstacle = [6,3]

check_single_obstacle(obstacle)

***loop found [6 3]


True

In [278]:
obstacle = [7,6]

g = Grid(lines, obstacle)


In [279]:
g.step()
g.print_grid()
# g.loop_found

....#.....
.........#
..........
..#.......
.......#..
....N.....
.#..^.....
........#.
#.........
......#...


False

In [281]:
obstacle_list = []
for i in range(len(lines)):
  for j in range(len(lines[0])):
    print(f'checking {[i,j]}')
    obstacle = [i,j]
    if check_single_obstacle(obstacle):
      obstacle_list.append(obstacle)
      print(obstacle)

checking [0, 0]
***Exiting room
checking [0, 1]
***Exiting room
checking [0, 2]
***Exiting room
checking [0, 3]
***Exiting room
checking [0, 4]
***Exiting room
checking [0, 5]
***Exiting room
checking [0, 6]
***Exiting room
checking [0, 7]
***Exiting room
checking [0, 8]
***Exiting room
checking [0, 9]
***Exiting room
checking [1, 0]
***Exiting room
checking [1, 1]
***Exiting room
checking [1, 2]
***Exiting room
checking [1, 3]
***Exiting room
checking [1, 4]
***Exiting room
checking [1, 5]
***Exiting room
checking [1, 6]
***Exiting room
checking [1, 7]
***Exiting room
checking [1, 8]
***Exiting room
checking [1, 9]
***Exiting room
checking [2, 0]
***Exiting room
checking [2, 1]
***Exiting room
checking [2, 2]
***Exiting room
checking [2, 3]
***Exiting room
checking [2, 4]
***Exiting room
checking [2, 5]
***Exiting room
checking [2, 6]
***Exiting room
checking [2, 7]
***Exiting room
checking [2, 8]
***Exiting room
checking [2, 9]
***Exiting room
checking [3, 0]
***Exiting room
checking

In [282]:
obstacle_list

[[6, 3], [7, 6], [7, 7], [8, 1], [8, 3], [9, 7]]

# now with the actual data

In [283]:
lines = read_input(input_filename)


In [286]:
obstacle_list = []
for i in range(len(lines)):
  for j in range(len(lines[0])):
    # print(f'checking {[i,j]}')
    obstacle = [i,j]
    if check_single_obstacle(obstacle):
      obstacle_list.append(obstacle)
      print(obstacle)

***loop found [4 8]
[4, 8]
***loop found [ 4 17]
[4, 17]
***loop found [ 4 18]
[4, 18]
***loop found [ 5 46]
[5, 46]
***loop found [ 5 52]
[5, 52]
***loop found [ 5 61]
[5, 61]
***loop found [ 5 64]
[5, 64]
***loop found [ 5 65]
[5, 65]
***loop found [ 5 69]
[5, 69]
***loop found [ 5 72]
[5, 72]
***loop found [7 7]
[7, 7]
***loop found [8 7]
[8, 7]
***loop found [ 8 45]
[8, 45]
***loop found [ 8 70]
[8, 70]
***loop found [ 8 71]
[8, 71]
***loop found [ 8 73]
[8, 73]
***loop found [ 8 74]
[8, 74]
***loop found [ 8 83]
[8, 83]
***loop found [ 8 84]
[8, 84]
***loop found [ 8 86]
[8, 86]
***loop found [ 8 89]
[8, 89]
***loop found [ 8 93]
[8, 93]
***loop found [ 9 45]
[9, 45]
***loop found [ 9 72]
[9, 72]
***loop found [ 9 93]
[9, 93]
***loop found [10  7]
[10, 7]
***loop found [11  7]
[11, 7]
***loop found [11 45]
[11, 45]
***loop found [12  7]
[12, 7]
***loop found [12 19]
[12, 19]
***loop found [12 29]
[12, 29]
***loop found [12 30]
[12, 30]
***loop found [12 34]
[12, 34]
***loop found 

In [287]:
len(obstacle_list)

2162